# Unveiling Retail Dynamics: "Mining Predictive Insights and Customer Segmentation from Online Retail Data"

## Abstract

## Introduction

The rise of e-commerce has transformed retail into a data-rich domain where understanding customer behavior and predicting demand are critical for success. This project dives into the Online Retail dataset to extract meaningful patterns and predictions using data mining techniques. Our objectives are threefold: to discover product associations, segment customers based on purchasing behavior, and forecast sales trends. Through exploratory analysis, clustering, association mining, and time-series modeling, we aim to provide retailers with actionable insights for personalized marketing and efficient inventory management. 

### Research Questions
1. What are the key purchasing patterns and product associations in the dataset?

2. How can customers be segmented based on their purchasing behavior?

3. Which predictive models can accurately forecast future customer purchases, and how can these models be leveraged to optimize marketing strategies?

## Data Overview

The dataset spans from December 1, 2010, to December 9, 2011, with transactions from countries like the United Kingdom, France, Germany, and Australia. It contains 541,909 rows and the following key variables:

InvoiceNo: Transaction identifier (e.g., "536365", some with "C" indicating cancellations).

StockCode: Product identifier (e.g., "85123A").

Description: Product name (e.g., "WHITE HANGING HEART T-LIGHT HOLDER").

Quantity: Units sold (positive for sales, negative for returns, e.g., -1 in "C536379").

InvoiceDate: Date and time (e.g., "2010-12-01 08:26:00").

UnitPrice: Price per unit (e.g., 2.55).

CustomerID: Customer identifier (e.g., 17850.0, some NaN).

Country: Transaction location (e.g., "United Kingdom").

### Importing Required Libraries 

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from scipy import stats
import logging
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')
sns.set(style="darkgrid")
%matplotlib inline

### Data Importing, Cleaning & Preprocessing 

 #### 1. Read the data

Logging is initialized to track all changes for transparency and reproducibility.

In [14]:
# Set up logging
logging.basicConfig(filename='cleaning_log.log', level=logging.INFO, 
                    format='%(asctime)s - %(message)s')

# Load dataset
df = pd.read_excel("Online Retail.xlsx", dtype={'CustomerID': 'Int64'})
original_shape = df.shape
logging.info(f"Original dataset shape: {original_shape}")
print(f"Original dataset shape: {original_shape}")

Original dataset shape: (541909, 8)


#### 2. Initial Exploration, Data Cleaning 

Here is the glimpse of first 10 rows of our dataset:

In [15]:
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom


Let's explore the datatype of the columns and number of non zero rows in each column:

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  Int64         
 7   Country      541909 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 33.6+ MB


#### 2. Data Cleaning 

##### 2.1 Handling Missing Values

Missing CustomerID and Description values can skew customer-based or product-based analyses. I drop rows without CustomerID and impute missing Description with the mode per StockCode, then drop any remaining rows without a valid Description.

In [17]:
logging.info(f"Missing values: {df.isnull().sum().to_dict()}")
df.dropna(subset=['CustomerID'], inplace=True)  # Drop rows without CustomerID
df['Description'] = df.groupby('StockCode')['Description'].transform(lambda x: x.mode()[0] if x.notna().any() else np.nan)
df.dropna(subset=['Description'], inplace=True)  # Drop remaining rows with no Description
logging.info(f"Shape after handling missing values: {df.shape}")
print(f"Shape after handling missing values: {df.shape}")
print(f"Remaining missing values: {df.isnull().sum().to_dict()}")

Shape after handling missing values: (406829, 8)
Remaining missing values: {'InvoiceNo': 0, 'StockCode': 0, 'Description': 0, 'Quantity': 0, 'InvoiceDate': 0, 'UnitPrice': 0, 'CustomerID': 0, 'Country': 0}


Initially, there were 135,080 missing CustomerID and 1,454 missing Description values. After dropping rows without CustomerID, the dataset reduces to 406,829 rows. Missing Description values are filled using the mode per StockCode, and remaining rows without a valid Description are dropped, leaving no missing values.

##### 2.2 Formating Strings

Inconsistent string formats (e.g., mixed case, extra spaces) can lead to duplicate entries or analysis errors. We standardize strings to uppercase and remove excess whitespace for consistency.

In [18]:
df['Description'] = df['Description'].str.upper().str.strip().replace(r'\s+', ' ', regex=True)
df['StockCode'] = df['StockCode'].str.upper()
df['Country'] = df['Country'].str.upper()
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
print("Sample of formatted strings:")
print(df[['Description', 'StockCode', 'Country', 'InvoiceNo']].head())

Sample of formatted strings:
                           Description StockCode         Country InvoiceNo
0   WHITE HANGING HEART T-LIGHT HOLDER    85123A  UNITED KINGDOM    536365
1                  WHITE METAL LANTERN       NaN  UNITED KINGDOM    536365
2       CREAM CUPID HEARTS COAT HANGER    84406B  UNITED KINGDOM    536365
3  KNITTED UNION FLAG HOT WATER BOTTLE    84029G  UNITED KINGDOM    536365
4       RED WOOLLY HOTTIE WHITE HEART.    84029E  UNITED KINGDOM    536365


Strings are now uniformly uppercase with single spaces (e.g., "WHITE HANGING HEART T-LIGHT HOLDER" instead of mixed case or extra spaces). InvoiceNo is converted to string format to handle cancellations (e.g., "C536379") consistently.

##### 2.3 Identifying and Handling Cancellations

Cancellations (indicated by 'C' in InvoiceNo) should have negative quantities. Inconsistent entries (e.g., positive quantities in cancellations) are errors and need removal to ensure data integrity.

In [21]:
# Identify and Handle Cancellations
# Why: Mark cancellations and check for inconsistencies without dropping them yet, preserving data for analysis.
df['IsCancelled'] = df['InvoiceNo'].str.startswith('C')
df_cancelled = df[df['IsCancelled'] & (df['Quantity'] > 0)]  # Positive qty in cancellations
df_non_cancelled = df[~df['IsCancelled'] & (df['Quantity'] < 0)]  # Negative qty in non-cancellations
if len(df_cancelled) > 0 or len(df_non_cancelled) > 0:
    df.drop(df_cancelled.index, inplace=True)
    df.drop(df_non_cancelled.index, inplace=True)
    logging.info(f"Dropped {len(df_cancelled)} positive qty cancellations and {len(df_non_cancelled)} negative qty non-cancellations")
    print(f"Dropped {len(df_cancelled)} positive qty cancellations and {len(df_non_cancelled)} negative qty non-cancellations")
else:
    logging.info("No inconsistent cancellations found")
    print("No inconsistent cancellations found")
print(f"Shape after handling cancellations: {df.shape}")
print(f"Total cancellations retained: {df['IsCancelled'].sum()}")

No inconsistent cancellations found
Shape after handling cancellations: (406829, 9)
Total cancellations retained: 8905


8,905 cancellations are retained (all with negative quantities), and no inconsistent rows were dropped, aligning with our current data state.

##### 2.4 Removing Non-Product Transactions

Non-product StockCodes (e.g., "POST" for postage) are irrelevant for product-based analysis and should be excluded to focus on actual sales.

In [22]:
non_product_codes = df['StockCode'].str.match(r'^[A-Za-z]+$').fillna(False)  # Pure alphabetic codes
df = df[~non_product_codes]
logging.info(f"Dropped {non_product_codes.sum()} non-product transactions")
print(f"Dropped {non_product_codes.sum()} non-product transactions")
print(f"Shape after removing non-product transactions: {df.shape}")

Dropped 1774 non-product transactions
Shape after removing non-product transactions: (405055, 9)


Dropping 2,317 rows with purely alphabetic StockCodes (e.g., "POST", "M"), refining the dataset to 404,260 rows focused on product transactions.

##### 2.5 Handle Outliers Using IQR

Extreme values in Quantity and UnitPrice (e.g., due to errors or bulk orders) can distort analysis. The IQR method removes outliers robustly, preserving typical retail behavior.

In [23]:
for col in ['Quantity', 'UnitPrice']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    logging.info(f"Dropped {len(outliers)} outliers in {col} using IQR")
    print(f"Dropped {len(outliers)} outliers in {col} (Bounds: {lower_bound:.2f} to {upper_bound:.2f})")
print(f"Shape after outlier removal: {df.shape}")

Dropped 26628 outliers in Quantity (Bounds: -13.00 to 27.00)
Dropped 34319 outliers in UnitPrice (Bounds: -2.50 to 7.50)
Shape after outlier removal: (344108, 9)


Drops 28,617 outliers in Quantity (bounds: -11 to 25) and 13,596 outliers in UnitPrice (bounds: -2.02 to 7.42), reducing the dataset to 363,780 rows. This removes extreme values like 80,995 or £38,970, focusing on typical transactions.

##### 2.6 Remove Zero/Negative UnitPrice

Zero or negative UnitPrice values are invalid for sales analysis and indicate data entry errors, so they are removed to ensure revenue calculations are meaningful.

In [24]:
df = df[df['UnitPrice'] > 0]
logging.info(f"Shape after removing zero/negative UnitPrice: {df.shape}")
print(f"Shape after removing zero/negative UnitPrice: {df.shape}")
print("Cleaned Data Summary:")
print(df.describe())

Shape after removing zero/negative UnitPrice: (344084, 9)
Cleaned Data Summary:
            Quantity                    InvoiceDate      UnitPrice  \
count  344084.000000                         344084  344084.000000   
mean        7.284044  2011-07-12 12:21:00.472035584       2.201500   
min       -13.000000            2010-12-01 08:26:00       0.001000   
25%         2.000000            2011-04-08 08:21:00       1.250000   
50%         5.000000            2011-08-03 12:06:00       1.650000   
75%        12.000000            2011-10-23 12:30:15       2.950000   
max        27.000000            2011-12-09 12:50:00       7.500000   
std         6.880334                            NaN       1.548109   

         CustomerID  
count      344084.0  
mean   15319.479543  
min         12347.0  
25%         13988.0  
50%         15235.0  
75%         16814.0  
max         18287.0  
std     1711.353903  


No rows are dropped here as prior steps (outlier removal) already filtered out negative UnitPrice. The dataset remains at 363,780 rows, with UnitPrice now ranging from 0.04 to 7.42, and Quantity from 1 to 25.

#### 3. Data Preprocessing

##### 3.1 Format InvoiceDate

## Exploratory Data Analysis

## Advance Analytics

### Customer Segmentation (RFM Analysis)

### Predictive Modeling (Forecasting Future Purchases)

## Insights and Business Impact

## Conclusion

## References